Importing Earth Engine Python API library and geemap for visualisation

In [1]:
import ee
import geemap

c:\Users\NIKHIL\anaconda3\envs\nikhil_geospatial\Lib\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Authenticate and initialize the Earth Engine service

In [2]:
ee.Authenticate()
ee.Initialize(project = 'ee-my-nikhil')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Importing parcels file from Earth Engine Assets Catalog after uploading 

In [3]:
roi = ee.FeatureCollection('projects/ee-my-nikhil/assets/parcels')

Linking Cloud Score + cs_cdf band to sentinel L2A Lcollection, 

Filtering the Image Collection to ROI and Date, 

Cloud Masking with cs_cdf band of cloud Score + 

In [4]:
cs = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterDate('2025-05-01', '2025-09-24')\
    .filterBounds(roi)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .select(['B.*'])\
    .linkCollection(cs, ['cs_cdf'])

def cloud_masking(image):
    return image.updateMask(image.select(['cs_cdf']).gte(0.60))

image = s2.map(cloud_masking)

checking original nominal scale (resolution) of sentinel

In [5]:
display(image.first().select(['B4']).projection())

function to increase the resolutoin with resampling (bilinear) and reprojecting,

Applying resampling function to image collection

check the nominal scale after resampling

In [6]:
def resampling(image):

    return image.resample('bilinear').reproject(
        'EPSG: 32644',
        None,
        1
    )

resampled = image.map(resampling)
display(resampled.first().select(['B4']).projection().nominalScale())


Creating a Medain Composite of resampled Image and applying scale value of 0.0001 to get actual reflactance values

In [7]:
median_composite = resampled.median().multiply(0.0001).clip(roi)
display(median_composite)

Deriving NDVI from median composite

In [8]:
ndvi = median_composite.normalizedDifference(['B8', 'B4']).rename('ndvi')

Fixing the threshold value based on individual pixel inspection after mutiple times of trial and error

creating binary class with 1 = vegetated and 0 = non vegetated pixels

In [9]:
threshold = 0.40

ndviClasses = ndvi.where(ndvi.lt(threshold), 0)\
            .where(ndvi.gte(threshold), 1)

Zonal Stats to get mean values for NDVI binary class

In [10]:
zonal_stats = ndviClasses.reduceRegions(
  collection = roi, 
  reducer = ee.Reducer.mean(),
  scale = 1
);

display(zonal_stats)

Mapping the function to add canopy height percentage and status of parcel land

In [11]:
def status(feature):

    canopy = ee.Number(feature.get('mean')).multiply(100)
    status = canopy.lt(10).multiply(1)

    statusText = ee.Algorithms.If(status.eq(1), 'Accepted', 'Rejected')

    return feature.set({

        'canopy_cover': canopy,
        'status': statusText
    })

    

final = zonal_stats.map(status)

Exporting the final FeatureCollection with canopy height and status

In [12]:
task = ee.batch.Export.table.toDrive(
    collection=final, description='parcels_status_python_GEE', fileFormat='GeoJSON'
)
task.start()

Visualise the Map Interactively

In [13]:
import geemap

m = geemap.Map()

m.centerObject(roi, 20)

m.add_layer(roi)
m.add_layer(ndviClasses, {
  'min': 0,
  'max': 1,
  'palette': ['ffe595', '#008000']
}, 'Binary classification')


display(m)

Map(center=[21.84673095422765, 80.42928627463107], controls=(WidgetControl(options=['position', 'transparent_b…